In [1]:
# Concurrency. asyncio.

In [2]:
# In the last lecture we seen how corotines and threds can help us by speeding up the running
# time of a scrpt. However threds have a drawback, they can be canceled if the raise a stoping
# point, to protect the important resources.

In [3]:
# Now we will tak a look at coroutines with asycio and aiohttp.

In [4]:
# Firstly let's try load our list of ids.
import json
albums_id = json.load(open('albums_ids.json', 'r'))

In [5]:
albums_id

['6VH2op0GKIl3WNTbZmmcmI',
 '23FJTTzUIUjhmimOE2CTX2',
 '0kTe1sQd9yhDsdG2Zth7X6',
 '4wExFfncaUIqSgoxnqa3Eh',
 '3otvl9PN3kOgk5uwAh1CBL',
 '56G9UnPmRifbubzPDJfAyd',
 '228mANuRrV20jS5DCA0eER',
 '5WRmchevXvk5eXPT0gEuTL',
 '1jCYuXr0Ujke24z1ymBr5U',
 '1W5CtQ7Ng0kP3lXyz7PIT2',
 '4wVHvawxZy52Oemd4sGyUm',
 '0ui4S0TZghkf1d1Wz0oWpk',
 '6vSiY2OVanKKforfEOT2PD',
 '3uhD8hNpb0m3iIZ18RHH5u',
 '3xiPkaTzfC48CcsXYgz5v0',
 '1lZahjeu4AhPkg9JARZr5F',
 '0ovKDDAHiTwg4AEjKdgdWo',
 '4Q7cPyiP8cMIlUEHAqeYfd',
 '26tH0kjUhkxBEd3ipGkx3Y',
 '7gS8ozSkvPW3VBPLnXOZ7S']

In [6]:
# Now as the last time we must autentificate ourselves on spotify.
import requests

In [7]:
# Now we need to setup the CLIENT_ID and the CLIENT_SECRET
CLIENT_ID = '1e8df497a8254e4b8ee14e464f164d71'
CLIENT_SECRET = '6d83175281d1495db717693dcbbf59fa'

In [8]:
# To get the data from spotify we must firstly autetithicate ourselves, let's define the 
# Autehtification url
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [9]:
# Now we can send the aiutetification request
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

In [10]:
# Now we must use the json funtion to convert the response into json format
auth_response_data = auth_response.json()

In [11]:
# Now we can get the accese token to get the data
access_token = auth_response_data['access_token']

In [12]:
# Let's extract the headers
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [24]:
TRACKS_NAMES = []
BASE_URL = 'https://api.spotify.com/v1/'

In [14]:
# Now we need a function that will return our tracks from the album.
# But firstly let's import the asyncio and aiohttp

In [15]:
import asyncio
import aiohttp

In [16]:
# of course we must firstly download aiohttp

In [141]:
!pip install aiohttp

In [142]:
!pip install nest_asyncio

In [17]:
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()

In [18]:
# Now we must define a async function that will add all Led Zepelin's tracks
async def get_track(album_id):
    # Adding 2 global variables
    global BASE_URL, TRACKS_NAMES
    async with aiohttp.request('get', BASE_URL + 'albums/' + album_id + '/tracks', 
                               headers=headers) as r:
        data = json.loads((await r.read()).decode())['items']
        # Adding the tracks to the global variable
        TRACKS_NAMES.extend([track['name'] for track in data])

In [19]:
# 'async' is a special key word from the asuncion library.
# It allows to define the asyncron functions.

In [20]:
# Now we must define a general function what will create the async tasks.
async def get_tracks(albums_ids):
    loop = asyncio.get_event_loop()
    to_do = [get_track(album_id) for album_id in sorted(albums_ids)]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)

In [25]:
# Now let's run the main function.
# For this we will use the asyncio.run function, passing the main function and it's arguments.
asyncio.run(get_tracks(albums_id))

In [26]:
# Let's look at the added tracks names
TRACKS_NAMES

['The Song Remains the Same - Remaster',
 'The Rain Song - Remaster',
 'Over the Hills and Far Away - Remaster',
 'The Crunge - Remaster',
 'Dancing Days - Remaster',
 "D'yer Mak'er - Remaster",
 'No Quarter - Remaster',
 'The Ocean - Remaster',
 'The Song Remains the Same - Guitar Overdub Reference Mix',
 'The Rain Song - Mix Minus Piano',
 'Over the Hills and Far Away - Guitar Mix Backing Track',
 'The Crunge - Rough Mix - Keys Up',
 'Dancing Days - Rough Mix with Vocal',
 'No Quarter - Rough Mix with JPJ Keyboard Overdubs - No Vocal',
 'The Ocean - Working Mix',
 'Custard Pie - 1990 Remaster',
 'The Rover - 1993 Remaster',
 'In My Time of Dying - 1990 Remaster',
 'Houses of the Holy - 1990 Remaster',
 'Trampled Under Foot - 1990 Remaster',
 'Kashmir - 1990 Remaster',
 'In the Light - 1990 Remaster',
 'Bron-Yr-Aur - 1993 Remaster',
 'Down by the Seaside - 1993 Remaster',
 'Ten Years Gone - 1990 Remaster',
 'Night Flight - 1993 Remaster',
 'The Wanton Song - 1990 Remaster',
 'Boogie w

In [27]:
# How many tracks do we have?
len(TRACKS_NAMES)

279

In [30]:
# Now let's see how much it takes to run our task with asyncio

In [31]:
%%timeit -n 1 -r 1
asyncio.run(get_tracks(albums_id))

229 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [32]:
# 229 ms. good job, even fuster than with the futures.